# Initialization

In [ ]:
import sys
sys.path.append('..')
from core.utilities.notebook_setups.frame_tools import *

SET = 'MOM'
URL = "https://www.17lands.com/tier_list/ad796ccaf0cc4be08511b0afd7a91a3c"
USER = "Alex"
DATES = ["230415", "230420", "230421", "230425", "230426"]

data_manager, set_data = set_up(log_lvl=LogLvl.ERROR, target_set=SET)

In [ ]:
agg = TierAggregator(SET)
tiers = [TierList.load(f"{SET}-{USER}-{date_str}.tier") for date_str in DATES]
for i in range(len(tiers)):
    tier = tiers[i]
    tier.user = f"{tier.creation_time.date()}"
    tier.tiers.index.name = tier.user
    agg.add_tier(tier)
    
x = DATES[-1]
most_recent = f"20{x[0:2]}-{x[2:4]}-{x[4:6]}"

# Monkey patch in the most recent tier update information, until this is further developped.
grade_cols = list(agg.tier_frame.columns)[5:-6]
agg._tier_frame['Last Update'] = agg._tier_frame[grade_cols[-1]] - agg._tier_frame[grade_cols[-2]]

## Tier List History

This displays the contents of the tierlists, with each row containing information about a card. On the left, there's information about the cards, along with a hoverable image. The `BO1` and `BO3` columns are how well the card is performing the the respective queue, calculated similarly to https://www.limitedgrades.com/mom.

The columns which are dates are all different version of Chord O' Calls' tier lists. The `2023-04-15` column has grades taken from the set review spreadsheet, while later dates are taken from 17Lands.  
(https://www.17lands.com/tier_list/ad796ccaf0cc4be08511b0afd7a91a3c).

`mean`, `max` and `min` ignore the `BO1` and `BO3` column, so shows the max and minimum the acard's been rated in the tierlist. `range` is the difference between `min` and `max`. `std` and `dist` are hold-overs from code this is based on for comparing different user's tier lists. Their values, along with `mean` likely don't have much value. `Last Update` shows the card's grade change between the previous and current tier list.

To simplify calculations, letter grades were converted to numbers. 12 is an A+, and 0 is an F. Sideboard or ungraded cards are given a null value.
```
12 -> A+
11 -> A
10 -> A-
9  -> B+
8  -> B
7  -> B-
6  -> C+
5  -> C
4  -> C-
3  -> D+
2  -> D
1  -> D-
0  -> F
```

### Top Cards

Below is a list of the top 3 uncomons and 5 commons for each colour, based on the most recent version of the tier list. There's no defined way of breaking ties for multiple cards that have the same grade.

In [ ]:
agg.top_picks(most_recent, style=True)

### Updated Cards
Any cards which had grades changed in the last Tier List update, ordered from most inmproved to least improved.

In [ ]:
updated_cards = agg.tier_frame[agg.tier_frame['Last Update'] != 0].sort_values('Last Update', ascending=False)
agg.style_frame(updated_cards)

### All Cards
This is a list of all cards in the tier list, orderd by the grades for the most recent date.

In [ ]:
agg.suborder_by_rarity(most_recent, style=True)

In [ ]:
raise Exception("Stopping Auto-Execute")

# Get Tier List
Run this to get today's Tier List.

In [ ]:
current_tier = TierList.from_link(URL, USER, SET)
current_tier.save()

Code used to generate this can be found here: https://github.com/ZacharyMN709/DraftMetaReporter